In [1]:
require 'dataset-mnist'

-- geometry: width and height of input images
geometry = {32,32}
numChannels = 1

nbTestingPatches = 10000

-- load test set
testData = mnist.loadTestSet(nbTestingPatches, geometry)
labels = torch.Tensor(nbTestingPatches)
for i = 1,nbTestingPatches do
     local sample = testData[i]
     local _,target = sample[2]:clone():max(1)
     target = target:squeeze()
     labels[i] = target
end

-- do global normalization
testData:normalizeGlobal()


print(testData:size())
print(testData.data:mean())
print(testData.data:std())

<mnist> done	


10000	


-2.0779814142286e-12	


1.0000000001234	


In [2]:
require 'nn'
--modelpath = "reduced-train-mnist-model.lua"
--local model = nn.Sequential()
--model:add(dofile(modelpath))
model1 = torch.load('mnist.baseline.net.final')

In [3]:
print(model1)

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> output]
  (1): nn.SpatialConvolutionMM(1 -> 32, 5x5)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(3,3,3,3)
  (4): nn.SpatialConvolutionMM(32 -> 64, 5x5)
  (5): nn.ReLU
  (6): nn.SpatialMaxPooling(2,2,2,2)
  (7): nn.Reshape(256)
  (8): nn.Linear(256 -> 200)
  (9): nn.ReLU
  (10): nn.Linear(200 -> 10)
  (11): nn.LogSoftMax
}
{
  gradInput : DoubleTensor - size: 100x1x32x32
  modules : 
    {
      1 : 
        nn.SpatialConvolutionMM(1 -> 32, 5x5)
        {
          dH : 1
          dW : 1
          nOutputPlane : 32
          output : DoubleTensor - size: 100x32x28x28
          gradInput : DoubleTensor - size: 100x1x32x32
          kW : 5
          finput : DoubleTensor - size: 100x25x784
          kH : 5
          nInputPlane : 1
          weight : DoubleTensor - size: 32x25
          padW : 0
          gradWeight : DoubleTensor - size: 32x25
          bias : DoubleTensor - size: 3

      indices : DoubleTensor - size: 100x64x2x2
          padW : 0
          ceil_mode : false
          kW : 2
          padH : 0
          gradInput : DoubleTensor - size: 100x64x5x5
        }
      7 : 
        nn.Reshape(256)
        {
          batchsize : LongStorage - size: 2
          size : LongStorage - size: 1
          output : DoubleTensor - size: 100x256
          gradInput : DoubleTensor - size: 100x64x2x2
          nelement : 256
          _gradOutput : DoubleTensor - empty
          _input : DoubleTensor - empty
        }
      8 : 
        nn.Linear(256 -> 200)
        {
          gradBias : DoubleTensor - size: 200
          weight : DoubleTensor - size: 200x256
          bias : DoubleTensor - size: 200
          gradInput : DoubleTensor - size: 100x256
          output : DoubleTensor - size: 100x200
          gradWeight : DoubleTensor - size: 200x256
          addBuffer : DoubleTensor - size: 100
        }
      9 : 
        nn.ReLU
        {
          inplace : fal

In [4]:
-- make a data batch
batchSize = 256
local inputs = torch.Tensor(batchSize,1,32,32)
local targets = torch.Tensor(batchSize)
local k = 1
for i = 1,batchSize do
     -- load new sample
     local sample = testData[i]
     local input = sample[1]:clone()
     local _,target = sample[2]:clone():max(1)
     target = target:squeeze()
     inputs[k] = input
     targets[k] = target
     k = k + 1
end
x = inputs:clone()
y = targets:clone()

In [5]:
print(x:size())
print(y:size())

 256
   1
  32
  32
[torch.LongStorage of size 4]

 256
[torch.LongStorage of size 1]



In [43]:
function hessianPowerMethod(inputs,targets,param,gradParam, delta, filepath, modelpath) 
    local maxIter = 20
    local epsilon = 10e-8 -- for numerical differentiation to get Hd 
    local criterion = nn.ClassNLLCriterion()
    local model_a = model1:clone()
    local model_b = model1:clone()
    local d = torch.randn(param:size()) --need to check
    print(d:size())
    
    local d_old = d*10; 
    local param_new_a,gradParam_eps_a = model_a:getParameters() --I need to do this
    local param_new_b,gradParam_eps_b = model_b:getParameters() --I need to do this
    -- in order to reflect loading a new parameter set
    local numIters = 0
    while torch.norm(d - d_old) > delta and numIters < maxIter do
        numIters = numIters+1
        print(numIters) --comment this out
        param_new_a:copy(param + d * epsilon)
        param_new_b:copy(param - d * epsilon)

        --reset gradients
        gradParam_eps_a:zero()
        gradParam_eps_b:zero()
        
        epsilon = 2*torch.sqrt(10e-7)*(1 + torch.norm(param))/torch.norm(d)


        --feedforward and backpropagation
        local outputs = model_a:forward(inputs)
        --local outputs_b = model_b:forward(inputs)
        local f = criterion:forward(outputs, targets)
        local df_do = criterion:backward(outputs, targets)
        model_a:backward(inputs, df_do) --gradParams_eps should be updated here 
        
        local outputs = model_b:forward(inputs)
        --local outputs_b = model_b:forward(inputs)
        local f = criterion:forward(outputs, targets)
        local df_do = criterion:backward(outputs, targets)
        model_b:backward(inputs, df_do) --gradParams_eps should be updated here 

        Hd = (gradParam_eps_a - gradParam_eps_b) / (2*epsilon)
        norm_Hd = torch.norm(Hd)
        -- normalize the resultant vector to a unit vector
        -- for the next iteration
        d_old = d
        d = Hd / norm_Hd
        print(torch.norm(d-d_old))
    end
    Hd = (gradParam_eps_a - gradParam_eps_b) / (2*epsilon)
    
    lambda = torch.dot(d, Hd)
    converged = true
    if numIters == maxIters
        then converged = false
    end
    return d , Hd, lambda, iConverged
end

In [73]:
-- usage example
local param, gradParam = model1:getParameters()
filepath = ""; modelpath = ""; delta = 5e-2
v, Hv, m, converged = hessianPowerMethod(x,y,param,gradParam, delta, filepath, modelpath) 


 105506
[torch.LongStorage of size 1]



1	


324.18079995839	
2	


0.25020942134123	
3	


0.23593859559132	
4	


...

15	


0.065280400137219	
16	


0.055746442460998	
17	


0.052703447122457	
18	


0.046244104282987	


In [74]:
print (m)
print (converged)

52.915274194191	
true	


In [75]:
function negativePowerMethod(inputs,targets,param,gradParam, delta, filepath, mEigVal, modelpath) 
    local maxIter = 20
    local epsilon = 10e-8 -- for numerical differentiation to get Hd 
    
    local criterion = nn.ClassNLLCriterion()
    local model_a = model1:clone()
    local model_b = model1:clone()
    local d = torch.randn(param:size()) --need to check
    print(d:size())
    
    local d_old = d*10; 
    local param_new_a,gradParam_eps_a = model_a:getParameters() --I need to do this
    local param_new_b,gradParam_eps_b = model_b:getParameters() --I need to do this
    -- in order to reflect loading a new parameter set
    local numIters = 0
    while torch.norm(d - d_old) > delta and numIters < maxIter do
        numIters = numIters+1
        print(numIters)
        param_new_a:copy(param + d * epsilon)
        param_new_b:copy(param - d * epsilon)

        --reset gradients
        gradParam_eps_a:zero()
        gradParam_eps_b:zero()
        
        epsilon = 2*torch.sqrt(10e-7)*(1 + torch.norm(param))/torch.norm(d)


        --feedforward and backpropagation
        local outputs = model_a:forward(inputs)
        --local outputs_b = model_b:forward(inputs)
        local f = criterion:forward(outputs, targets)
        local df_do = criterion:backward(outputs, targets)
        model_a:backward(inputs, df_do) --gradParams_eps should be updated here 
        
        local outputs = model_b:forward(inputs)
        --local outputs_b = model_b:forward(inputs)
        local f = criterion:forward(outputs, targets)
        local df_do = criterion:backward(outputs, targets)
        model_b:backward(inputs, df_do) --gradParams_eps should be updated here 

        Hd = (gradParam_eps_a - gradParam_eps_b) / (2*epsilon)
        norm_Hd = torch.norm(Hd)
        -- normalize the resultant vector to a unit vector
        -- for the next iteration
        d_old = d
        Md = d*mEigVal - Hd
        d = Md / torch.norm(Md)
        print(torch.norm(d-d_old))
    end
    lambda = torch.dot(d, Md)
    if numIters == maxIters
        then converged = false
    end
    return d , Md, Hd, lambda, converged
    --]]
end

In [82]:
-- usage example
local param, gradParam = model1:getParameters()
filepath = ""; modelpath = ""; delta = 1e-4
v, Mv, Hv, lambda, converged = negativePowerMethod(x,y,param,gradParam, delta, filepath, m, modelpath) 

 105506
[torch.LongStorage of size 1]



1	


323.24242853615	
2	


7.655164456167e-06	


In [83]:
print (lambda) -- largest eigenvalue of mI-H
print (m) 
-- sanity checks
print(m-lambda) -- smallest eigvalue of H, should be of less magitude than m
print(torch.dot(v,Hv)) --should  equal m-lambda

52.915271187676	
52.915274194191	
3.0065152145653e-06	
3.0049650400118e-06	
